In [25]:
! pip install opencv-python


In [26]:
! pip uninstall -y numpy scipy seaborn tensorflow opencv-python


Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: scipy 1.11.4
Uninstalling scipy-1.11.4:
  Successfully uninstalled scipy-1.11.4
Found existing installation: seaborn 0.13.2
Uninstalling seaborn-0.13.2:
  Successfully uninstalled seaborn-0.13.2
Found existing installation: tensorflow 2.16.2
Uninstalling tensorflow-2.16.2:
  Successfully uninstalled tensorflow-2.16.2
Found existing installation: opencv-python 4.9.0.80
Uninstalling opencv-python-4.9.0.80:
  Successfully uninstalled opencv-python-4.9.0.80


In [27]:
! pip install \
numpy==1.26.4 \
scipy==1.11.4 \
pandas==2.2.2 \
matplotlib==3.8.4 \
seaborn==0.13.2 \
opencv-python==4.9.0.80 \
tensorflow==2.16.2


  Using cached numpy-1.26.4-cp312-cp312-macosx_10_9_x86_64.whl.metadata (61 kB)
  Using cached scipy-1.11.4-cp312-cp312-macosx_10_9_x86_64.whl.metadata (60 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached opencv_python-4.9.0.80-cp37-abi3-macosx_10_16_x86_64.whl.metadata (20 kB)
  Using cached tensorflow-2.16.2-cp312-cp312-macosx_10_15_x86_64.whl.metadata (4.1 kB)
Using cached numpy-1.26.4-cp312-cp312-macosx_10_9_x86_64.whl (20.3 MB)
Using cached scipy-1.11.4-cp312-cp312-macosx_10_9_x86_64.whl (37.1 MB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)
Using cached opencv_python-4.9.0.80-cp37-abi3-macosx_10_16_x86_64.whl (55.7 MB)
Using cached tensorflow-2.16.2-cp312-cp312-macosx_10_15_x86_64.whl (259.7 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [seaborn]m4/5 [seaborn]ow]on]


In [1]:

import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import os

from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img 
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, SeparableConv2D, MaxPooling2D, Lambda
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2DTranspose, Conv2D, add, concatenate
from tensorflow.keras.layers import LeakyReLU, Activation, Reshape
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adam 
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau

2025-12-27 01:33:18.969319: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
attributes=pd.read_csv('list_attr_celeba.csv')
bboxes=pd.read_csv('list_bbox_celeba.csv')
partition=pd.read_csv('list_eval_partition.csv')
landmarks=pd.read_csv('list_landmarks_align_celeba.csv')

In [3]:
from pathlib import Path

# Resolve the CSV relative to the notebook location
base_dir = Path(__file__).resolve().parent if '__file__' in globals() else Path.cwd()
landmarks_path = base_dir / 'list_landmarks_align_celeba.csv'

if not landmarks_path.exists():
    raise FileNotFoundError(f"Missing file: {landmarks_path}")

landmarks_preview = pd.read_csv(landmarks_path)
landmarks_preview.head()

,image_id,lefteye_x,lefteye_y,righteye_x,righteye_y,nose_x,nose_y,leftmouth_x,leftmouth_y,rightmouth_x,rightmouth_y
0,000001.jpg,69,109,106,113,77,142,73,152,108,154
1,000002.jpg,69,110,107,112,81,135,70,151,108,153
2,000003.jpg,76,112,104,106,108,128,74,156,98,158
3,000004.jpg,72,113,108,108,101,138,71,155,101,151
4,000005.jpg,66,114,112,112,86,119,71,147,104,150


In [31]:
from pathlib import Path
import math
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

base_dir = Path(__file__).resolve().parent if '__file__' in globals() else Path.cwd()
candidate_dirs = [base_dir / 'img_align_celeba', base_dir / 'data' / 'img_align_celeba']
img_files = []
for folder in candidate_dirs:
    if folder.exists():
        img_files = sorted(folder.glob('*.jpg'))
        if img_files:
            break

if not img_files:
    print('No images found. Place sample JPGs in img_align_celeba/ or data/img_align_celeba/.')
else:
    images = img_files[:20]
    cols = 5
    rows = math.ceil(len(images) / cols)
    plt.figure(figsize=(cols * 3, rows * 3))
    for idx, img_path in enumerate(images):
        plt.subplot(rows, cols, idx + 1)
        plt.axis('off')
        plt.title(img_path.name, fontsize=8)
        plt.imshow(mpimg.imread(img_path))
    plt.tight_layout()

No images found. Place sample JPGs in img_align_celeba/ or data/img_align_celeba/.


In [4]:
landmarks_df=pd.read_csv('list_landmarks_align_celeba.csv')
landmarks_df.head()

,image_id,lefteye_x,lefteye_y,righteye_x,righteye_y,nose_x,nose_y,leftmouth_x,leftmouth_y,rightmouth_x,rightmouth_y
0,000001.jpg,69,109,106,113,77,142,73,152,108,154
1,000002.jpg,69,110,107,112,81,135,70,151,108,153
2,000003.jpg,76,112,104,106,108,128,74,156,98,158
3,000004.jpg,72,113,108,108,101,138,71,155,101,151
4,000005.jpg,66,114,112,112,86,119,71,147,104,150


In [33]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Locate data (include nested img_align_celeba/img_align_celeba case)
base_dir = Path(__file__).resolve().parent if '__file__' in globals() else Path.cwd()
extra_root = Path.home() / 'Downloads' / 'archive (5)' / 'img_align_celeba'
extra_nested = extra_root / 'img_align_celeba'
img_dir_candidates = [
    base_dir / 'img_align_celeba',
    base_dir / 'data' / 'img_align_celeba',
    extra_root,
    extra_nested,
]
img_dir = next((d for d in img_dir_candidates if d.exists()), None)
landmarks_path = base_dir / 'list_landmarks_align_celeba.csv'

print(f"Checking image dirs (exists?): {[str(d) + ' ✔' if d.exists() else str(d) + ' ✖' for d in img_dir_candidates]}")

if img_dir is None:
    print('No image directory found. Create img_align_celeba/ or data/img_align_celeba/ or update extra path.')
elif not landmarks_path.exists():
    print(f'Missing landmarks file: {landmarks_path}')
else:
    landmarks = pd.read_csv(landmarks_path)
    img_path = None
    landmark_row = None
    for _, row in landmarks.iterrows():
        candidate = img_dir / row['image_id']
        if candidate.exists():
            img_path = candidate
            landmark_row = row
            break

    if img_path is None:
        print(f'No matching image files found in {img_dir} for entries in landmarks CSV.')
    else:
        img = mpimg.imread(img_path)
        plt.figure(figsize=(6, 6))
        plt.imshow(img)
        keypoints = ['lefteye', 'righteye', 'nose', 'leftmouth', 'rightmouth']
        for key in keypoints:
            x = landmark_row[f'{key}_x']
            y = landmark_row[f'{key}_y']
            plt.scatter(x, y, c='red', s=30)
            plt.text(x + 2, y - 2, key, color='yellow', fontsize=8)
        plt.title(f'Landmarks for {img_path.name}')
        plt.axis('off')
        plt.tight_layout()

Checking image dirs (exists?): ['/Users/imac/DeepLearning2/img_align_celeba ✖', '/Users/imac/DeepLearning2/data/img_align_celeba ✖', '/Users/imac/Downloads/archive (5)/img_align_celeba ✔', '/Users/imac/Downloads/archive (5)/img_align_celeba/img_align_celeba ✔']
No matching image files found in /Users/imac/Downloads/archive (5)/img_align_celeba for entries in landmarks CSV.


In [5]:
bboxes_df=pd.read_csv('list_bbox_celeba.csv')
bboxes_df.head()

,image_id,x_1,y_1,width,height
0,000001.jpg,95,71,226,313
1,000002.jpg,72,94,221,306
2,000003.jpg,216,59,91,126
3,000004.jpg,622,257,564,781
4,000005.jpg,236,109,120,166


In [35]:
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Plot a bounding box using existing bboxes_df
globals_present = globals()
if 'bboxes_df' not in globals_present:
    print('bboxes_df not defined. Run the cell that loads list_bbox_celeba.csv first.')
else:
    base_dir = Path(__file__).resolve().parent if '__file__' in globals() else Path.cwd()
    img_candidates = [
        base_dir / 'img_align_celeba',
        base_dir / 'data' / 'img_align_celeba',
        Path.home() / 'Downloads' / 'archive (5)' / 'img_align_celeba',
        Path.home() / 'Downloads' / 'archive (5)' / 'img_align_celeba' / '000006.jpg',
    ]
    img_dir = next((d for d in img_candidates if d.exists()), None)
    print('Image dirs:', [f"{d}{' ✔' if d.exists() else ' ✖'}" for d in img_candidates])

    if img_dir is None:
        print('No image directory found. Update img_candidates to your path.')
    else:
        img_path = None
        bbox_row = None
        for _, row in bboxes_df.iterrows():
            candidate = img_dir / row['image_id']
            if candidate.exists():
                img_path = candidate
                bbox_row = row
                break

        if img_path is None:
            print(f'No matching image files found in {img_dir} for entries in bboxes_df.')
        else:
            img = mpimg.imread(img_path)
            fig, ax = plt.subplots(figsize=(6, 6))
            ax.imshow(img)
            rect = plt.Rectangle(
                (bbox_row['x_1'], bbox_row['y_1']),
                bbox_row['width'],
                bbox_row['height'],
                linewidth=2,
                edgecolor='cyan',
                facecolor='none'
            )
            ax.add_patch(rect)
            ax.set_title(f"Bounding box for {img_path.name}")
            ax.axis('off')
            plt.tight_layout()

Image dirs: ['/Users/imac/DeepLearning2/img_align_celeba ✖', '/Users/imac/DeepLearning2/data/img_align_celeba ✖', '/Users/imac/Downloads/archive (5)/img_align_celeba ✔', '/Users/imac/Downloads/archive (5)/img_align_celeba/000006.jpg ✖']
No matching image files found in /Users/imac/Downloads/archive (5)/img_align_celeba for entries in bboxes_df.


In [6]:
#reading the partiton files
partition_df=pd.read_csv('list_eval_partition.csv')
partition_df.head()

,image_id,partition
0,000001.jpg,0
1,000002.jpg,0
2,000003.jpg,0
3,000004.jpg,0
4,000005.jpg,0


In [7]:
partition_df['partition'].value_counts()

partition
0    162770
2     19962
1     19867
Name: count, dtype: int64

In [8]:
train_images=partition.query('partition==0')
valid_images=partition.query('partition==1')
test_images=partition.query('partition==2')

list the agtributes files 

In [9]:
atributes_df=pd.read_csv('list_attr_celeba.csv')
atributes_df.head()

,image_id,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,...,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
0,000001.jpg,-1,1,1,-1,-1,-1,-1,-1,-1,...,-1,1,1,-1,1,-1,1,-1,-1,1
1,000002.jpg,-1,-1,-1,1,-1,-1,-1,1,-1,...,-1,1,-1,-1,-1,-1,-1,-1,-1,1
2,000003.jpg,-1,-1,-1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,1,-1,-1,-1,-1,-1,1
3,000004.jpg,-1,-1,1,-1,-1,-1,-1,-1,-1,...,-1,-1,1,-1,1,-1,1,1,-1,1
4,000005.jpg,-1,1,1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,-1,-1,-1,1,-1,-1,1


In [11]:
import tensorflow as tf 
from tensorflow.keras import layers
from tensorflow import keras 
import numpy as np 
import matplotlib.pyplot as plt 
import os
discriminator=keras.Sequential([
    keras.Input(shape=(64,64,3)),
    layers.Conv2D(64,kernel_size=4,strides=2,padding="same"),
    layers.LeakyReLU(alpha=0.2),
    layers.Conv2D(128,kernel_size=4,strides=2,padding="same"),
    layers.LeakyReLU(alpha=0.2),
    layers.Conv2D(128,kernel_size=4,strides=2,padding="same"),
    layers.LeakyReLU(alpha=0.2),
    layers.Flatten(),
    layers.Dropout(0.2),
    layers.Dense(1,activation="sigmoid"),],
    name="discriminator",

    

)
discriminator.summary()




Model: "discriminator"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 32, 32, 64)     │         3,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_3 (LeakyReLU)       │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 16, 16, 128)    │       131,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_4 (LeakyReLU)       │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 8, 8, 128)      │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_5 (LeakyReLU)       │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │         8,193 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 404,801 (1.54 MB)

 Trainable params: 404,801 (1.54 MB)

 Non-trainable params: 0 (0.00 B)

Create the generator 

In [12]:
latent_dim=100 
generator=keras.Sequential(
    [
        keras.Input(shape=(latent_dim,)),
        layers.Dense(8*8*128),
        layers.Reshape((8,8,128)),
        layers.Conv2DTranspose(128,kernel_size=4,strides=2,padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(256,kernel_size=4,strides=2,padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(512,kernel_size=4,strides=2,padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(3,kernel_size=5,padding="same",activation="sigmoid"),

        

    ],
    name="generator",
)
generator.summary()

Model: "generator"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                 │ (None, 8192)           │       827,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose                │ (None, 16, 16, 128)    │       262,272 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_6 (LeakyReLU)       │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_1              │ (None, 32, 32, 256)    │       524,544 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_7 (LeakyReLU)       │ (None, 32, 32, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_2              │ (None, 64, 64, 512)    │     2,097,664 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_8 (LeakyReLU)       │ (None, 64, 64, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 64, 64, 3)      │        38,403 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,750,275 (14.31 MB)

 Trainable params: 3,750,275 (14.31 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
class GAN(keras.Model):
    def __init__(self,discriminator,generator,latent_dim):
        super(GAN,self).__init__()
        self.discriminator=discriminator
        self.generator=generator 
        self.latent_dim=latent_dim
    def compile(self,d_optimizer,g_optimizer,loss_fn):
        super(GAN,self).compile()
        self.d_optimizer=d_optimizer
        self.g_optimizer=g_optimizer 
        self.loss_fn=loss_fn
        self.d_loss_metric=keras.metrics.Mean(name="d_loss")
        self.g_loss_metric=keras.metrics.Mean(name="g_loss")
    @property 
    def metrics(self):
        return [self.d_loss_metric,self.g_loss_metric]
    def train_step(self,real_images):
        batch_size=tf.shape(real_images)[0]
        random_latent_vectors=tf.random.normal(shape=(batch_size,self.latent_dim))
        generated_images=self.generator(random_latent_vectors)
        combined_images=tf.concat([tf.ones(batch_size,1),tf.zeros((batch_size,1))],axis=0)
        labels=tf.concat([tf.ones((batch_size,1)),tf.zeros((batch_size,1))],axis=0)
        labels+=0.05*tf.random.uniform(tf.shape(labels))
        with tf.GradientTape() as tape:
            predictions=self.discriminator(combined_images)
            d_loss=self.loss_fn(labels,predictions)
        grads=tape.gradient(d_loss,self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(zip(grads,self.discriminator.trainable_weights))

        self.d_loss_metric.update_state(d_loss)
        self.g_loss_metric.update_state(g_loss)
        return {
            "d_loss" : self.d_loss_metric.result(),
            "g_loss":self.g_loss_metric.result(),
        }

         

